# api 호출을 이용한 데이터 수집(전사, 궁수, 해적)

In [ ]:
# 김수현 api key
"live_0d472109d07d1f4804875debf5c6b0e5bd5ee7e5a3c95d10227aa141a6bdbf6defe8d04e6d233bd35cf2fabdeb93fb0d"

In [ ]:
# 전사 직업군 정보 가져오기

import os
import aiohttp
import asyncio
import pandas as pd
from aiohttp import ClientSession

# 날짜 및 API 키 설정
date = "2025-04-15"
API_KEY = "live_4bb9fe2d9c5f88d6d4d0fae33b021fd47a586db57c1872bf6649cad9965e1621efe8d04e6d233bd35cf2fabdeb93fb0d"
HEADERS = {"x-nxopen-api-key": API_KEY}

# 대상 클래스 리스트 (수정 부탁드려요!)
target_classes = [
    "전사-히어로", "전사-팔라딘", "전사-다크나이트",
    "기사단-소울마스터", "기사단-미하일",
    "레지스탕스-블래스터", "레지스탕스-데몬슬레이어", "레지스탕스-데몬어벤져",
    "아란-전체 전직", "카이저-전체 전직", "아델-전체 전직", "초월자-제로"
]

# character 및 user 관련 endpoint 리스트
character_endpoints = [
    "item-equipment", "hyper-stat", "ability",
    "vmatrix", "hexamatrix", "symbol-equipment",
    "set-effect", "link-skill", "hexamatrix-stat"
]

user_endpoints = [
    "union", "union-raider", "union-artifact", "union-champion"
]

# JSON 요청 함수
async def fetch_json(session: ClientSession, url: str, params: dict):
    try:
        async with session.get(url, headers=HEADERS, params=params) as response:
            if response.status == 200:
                return await response.json()
            else:
                print(f"❌ 실패: {url} | 상태코드: {response.status}")
    except Exception as e:
        print(f"❌ 예외 발생: {url} | {e}")
    return {}

# ocid로 캐릭터 상세 데이터 가져오기
async def fetch_character_data(session: ClientSession, character):
    ocid_url = "https://open.api.nexon.com/maplestory/v1/id"
    ocid_params = {
        "character_name": character["nickname"],
        "world_name": character["world"]
    }
    ocid_data = await fetch_json(session, ocid_url, ocid_params)
    ocid = ocid_data.get("ocid")
    if not ocid:
        print(f"❗ocid 못 찾음 → 닉네임: {character['nickname']} | 월드: {character['world']}")
        return None

    if "-" in character["class"]:
        job, subclass = character["class"].split("-")
    else:
        job = subclass = character["class"]

    if subclass == "전체 전직":
        subclass = job

    result = {
        "nickname": character["nickname"],
        "world": character["world"],
        "level": character["level"],
        "rank": character["rank"],
        "job": job,
        "subclass": subclass
    }

    # character 관련 데이터 수집
    for ep in character_endpoints:
        url = f"https://open.api.nexon.com/maplestory/v1/character/{ep}"
        data = await fetch_json(session, url, {"ocid": ocid, "date": date})
        result[ep] = data

    # user 관련 데이터 수집(유니온)
    for ep in user_endpoints:
        url = f"https://open.api.nexon.com/maplestory/v1/user/{ep}"
        data = await fetch_json(session, url, {"ocid": ocid, "date": date})
        result[ep] = data

    return result

# 클래스별 랭킹 수집
async def gather_rankings_for_class(session, class_name):
    character_list = []
    for page in range(1, 6):  # 페이지 수 조절 가능
        params = {
            "date": date,
            "class": class_name,
            "page": page
        }
        data = await fetch_json(session, "https://open.api.nexon.com/maplestory/v1/ranking/overall", params)
        if not data or "ranking" not in data:
            break
        for entry in data["ranking"]:
            character_list.append({
                "nickname": entry.get("character_name"),
                "world": entry.get("world_name"),
                "level": entry.get("character_level"),
                "rank": entry.get("ranking"),
                "class": class_name
            })
        await asyncio.sleep(0.1)
    return character_list

# 전체 실행 함수
async def main():
    os.makedirs("warrior_json", exist_ok=True) # warrior_json: 폴더명(수정해서 사용해주세요!)
    async with ClientSession() as session:
        for class_name in target_classes:
            print(f"📥 {class_name} 수집 시작")
            character_list = await gather_rankings_for_class(session, class_name)
            results = []
            tasks = [fetch_character_data(session, char) for char in character_list]
            for i in range(0, len(tasks), 50):
                chunk = tasks[i:i + 50]
                chunk_results = await asyncio.gather(*chunk)
                results.extend([r for r in chunk_results if r])
                await asyncio.sleep(0.2)

            df = pd.DataFrame(results)
            safe_class_name = class_name.replace("-", "_")
            df.to_json(f"warrior_json/{safe_class_name}.json", orient="records", force_ascii=False) # warrior_json: 폴더명(수정해서 사용해주세요!)
            print(f"✅ 저장 완료: {safe_class_name}.json")

    return "완료!"

# 실행
await main()

📥 전사-히어로 수집 시작
✅ 저장 완료: 전사_히어로.json
📥 전사-팔라딘 수집 시작
✅ 저장 완료: 전사_팔라딘.json
📥 전사-다크나이트 수집 시작
✅ 저장 완료: 전사_다크나이트.json
📥 기사단-소울마스터 수집 시작
✅ 저장 완료: 기사단_소울마스터.json
📥 기사단-미하일 수집 시작
❌ 실패: https://open.api.nexon.com/maplestory/v1/id | 상태코드: 400
❗ocid 못 찾음 → 닉네임: 호위 | 월드: 베라
✅ 저장 완료: 기사단_미하일.json
📥 레지스탕스-블래스터 수집 시작
✅ 저장 완료: 레지스탕스_블래스터.json
📥 레지스탕스-데몬슬레이어 수집 시작
❌ 실패: https://open.api.nexon.com/maplestory/v1/id | 상태코드: 400
❗ocid 못 찾음 → 닉네임: 꼬마뎨슬 | 월드: 루나
✅ 저장 완료: 레지스탕스_데몬슬레이어.json
📥 레지스탕스-데몬어벤져 수집 시작
❌ 실패: https://open.api.nexon.com/maplestory/v1/id | 상태코드: 400
❗ocid 못 찾음 → 닉네임: Aroma | 월드: 엘리시움
✅ 저장 완료: 레지스탕스_데몬어벤져.json
📥 아란-전체 전직 수집 시작
✅ 저장 완료: 아란_전체 전직.json
📥 카이저-전체 전직 수집 시작
✅ 저장 완료: 카이저_전체 전직.json
📥 아델-전체 전직 수집 시작
✅ 저장 완료: 아델_전체 전직.json
📥 초월자-제로 수집 시작
✅ 저장 완료: 초월자_제로.json


'완료!'

In [ ]:
# 궁수 직업군 정보 가져오기

import os
import aiohttp
import asyncio
import pandas as pd
from aiohttp import ClientSession

# 날짜 및 API 키 설정
date = "2025-04-15"
API_KEY = "live_4bb9fe2d9c5f88d6d4d0fae33b021fd47a586db57c1872bf6649cad9965e1621efe8d04e6d233bd35cf2fabdeb93fb0d"
HEADERS = {"x-nxopen-api-key": API_KEY}

# 대상 클래스 리스트 (수정 부탁드려요!)
target_classes = [
    "궁수-보우마스터", "궁수-신궁", "궁수-패스파인더",
    "기사단-윈드브레이커", "레지스탕스-와일드헌터",
    "메르세데스-전체 전직", "카인-전체 전직"
]

# character 및 user 관련 endpoint 리스트
character_endpoints = [
    "item-equipment", "hyper-stat", "ability",
    "vmatrix", "hexamatrix", "symbol-equipment",
    "set-effect", "link-skill", "hexamatrix-stat"
]

user_endpoints = [
    "union", "union-raider", "union-artifact", "union-champion"
]

# JSON 요청 함수
async def fetch_json(session: ClientSession, url: str, params: dict):
    try:
        async with session.get(url, headers=HEADERS, params=params) as response:
            if response.status == 200:
                return await response.json()
            else:
                print(f"❌ 실패: {url} | 상태코드: {response.status}")
    except Exception as e:
        print(f"❌ 예외 발생: {url} | {e}")
    return {}

# ocid로 캐릭터 상세 데이터 가져오기
async def fetch_character_data(session: ClientSession, character):
    ocid_url = "https://open.api.nexon.com/maplestory/v1/id"
    ocid_params = {
        "character_name": character["nickname"],
        "world_name": character["world"]
    }
    ocid_data = await fetch_json(session, ocid_url, ocid_params)
    ocid = ocid_data.get("ocid")
    if not ocid:
        print(f"❗ocid 못 찾음 → 닉네임: {character['nickname']} | 월드: {character['world']}")
        return None

    if "-" in character["class"]:
        job, subclass = character["class"].split("-")
    else:
        job = subclass = character["class"]

    if subclass == "전체 전직":
        subclass = job

    result = {
        "nickname": character["nickname"],
        "world": character["world"],
        "level": character["level"],
        "rank": character["rank"],
        "job": job,
        "subclass": subclass
    }

    # character 관련 데이터 수집
    for ep in character_endpoints:
        url = f"https://open.api.nexon.com/maplestory/v1/character/{ep}"
        data = await fetch_json(session, url, {"ocid": ocid, "date": date})
        result[ep] = data

    # user 관련 데이터 수집(유니온)
    for ep in user_endpoints:
        url = f"https://open.api.nexon.com/maplestory/v1/user/{ep}"
        data = await fetch_json(session, url, {"ocid": ocid, "date": date})
        result[ep] = data

    return result

# 클래스별 랭킹 수집
async def gather_rankings_for_class(session, class_name):
    character_list = []
    for page in range(1, 6):  # 페이지 수 조절 가능
        params = {
            "date": date,
            "class": class_name,
            "page": page
        }
        data = await fetch_json(session, "https://open.api.nexon.com/maplestory/v1/ranking/overall", params)
        if not data or "ranking" not in data:
            break
        for entry in data["ranking"]:
            character_list.append({
                "nickname": entry.get("character_name"),
                "world": entry.get("world_name"),
                "level": entry.get("character_level"),
                "rank": entry.get("ranking"),
                "class": class_name
            })
        await asyncio.sleep(0.1)
    return character_list

# 전체 실행 함수
async def main():
    os.makedirs("archer_json", exist_ok=True) # archer_json: 폴더명(수정해서 사용해주세요!)
    async with ClientSession() as session:
        for class_name in target_classes:
            print(f"📥 {class_name} 수집 시작")
            character_list = await gather_rankings_for_class(session, class_name)
            results = []
            tasks = [fetch_character_data(session, char) for char in character_list]
            for i in range(0, len(tasks), 50):
                chunk = tasks[i:i + 50]
                chunk_results = await asyncio.gather(*chunk)
                results.extend([r for r in chunk_results if r])
                await asyncio.sleep(0.2)

            df = pd.DataFrame(results)
            safe_class_name = class_name.replace("-", "_")
            df.to_json(f"archer_json/{safe_class_name}.json", orient="records", force_ascii=False) # archer_json: 폴더명(수정해서 사용해주세요!)
            print(f"✅ 저장 완료: {safe_class_name}.json")

    return "완료!"

# 실행
await main()

📥 궁수-보우마스터 수집 시작
✅ 저장 완료: 궁수_보우마스터.json
📥 궁수-신궁 수집 시작
❌ 실패: https://open.api.nexon.com/maplestory/v1/id | 상태코드: 400
❗ocid 못 찾음 → 닉네임: 2Q4Q | 월드: 오로라
❌ 실패: https://open.api.nexon.com/maplestory/v1/id | 상태코드: 400
❗ocid 못 찾음 → 닉네임: 신궁자는중 | 월드: 루나
✅ 저장 완료: 궁수_신궁.json
📥 궁수-패스파인더 수집 시작
✅ 저장 완료: 궁수_패스파인더.json
📥 기사단-윈드브레이커 수집 시작
✅ 저장 완료: 기사단_윈드브레이커.json
📥 레지스탕스-와일드헌터 수집 시작
❌ 실패: https://open.api.nexon.com/maplestory/v1/id | 상태코드: 400
❗ocid 못 찾음 → 닉네임: 꾸쓰헌터 | 월드: 스카니아
❌ 실패: https://open.api.nexon.com/maplestory/v1/id | 상태코드: 400
❗ocid 못 찾음 → 닉네임: 봉민구 | 월드: 베라
✅ 저장 완료: 레지스탕스_와일드헌터.json
📥 메르세데스-전체 전직 수집 시작
✅ 저장 완료: 메르세데스_전체 전직.json
📥 카인-전체 전직 수집 시작
❌ 실패: https://open.api.nexon.com/maplestory/v1/id | 상태코드: 400
❗ocid 못 찾음 → 닉네임: 헤응늘 | 월드: 크로아
✅ 저장 완료: 카인_전체 전직.json


'완료!'

In [ ]:
# 해적 직업군 정보 가져오기

import os
import aiohttp
import asyncio
import pandas as pd
from aiohttp import ClientSession

# 날짜 및 API 키 설정
date = "2025-04-15"
API_KEY = "live_4bb9fe2d9c5f88d6d4d0fae33b021fd47a586db57c1872bf6649cad9965e1621efe8d04e6d233bd35cf2fabdeb93fb0d"
HEADERS = {"x-nxopen-api-key": API_KEY}

# 대상 클래스 리스트 (수정 부탁드려요!)
target_classes = [
    "해적-바이퍼", "해적-캡틴", "해적-캐논마스터",
    "기사단-스트라이커", "레지스탕스-메카닉",
    "엔젤릭버스터-전체 전직", "은월-전체 전직", "아크-전체 전직"
]

# character 및 user 관련 endpoint 리스트
character_endpoints = [
    "item-equipment", "hyper-stat", "ability",
    "vmatrix", "hexamatrix", "symbol-equipment",
    "set-effect", "link-skill", "hexamatrix-stat"
]

user_endpoints = [
    "union", "union-raider", "union-artifact", "union-champion"
]

# JSON 요청 함수
async def fetch_json(session: ClientSession, url: str, params: dict):
    try:
        async with session.get(url, headers=HEADERS, params=params) as response:
            if response.status == 200:
                return await response.json()
            else:
                print(f"❌ 실패: {url} | 상태코드: {response.status}")
    except Exception as e:
        print(f"❌ 예외 발생: {url} | {e}")
    return {}

# ocid로 캐릭터 상세 데이터 가져오기
async def fetch_character_data(session: ClientSession, character):
    ocid_url = "https://open.api.nexon.com/maplestory/v1/id"
    ocid_params = {
        "character_name": character["nickname"],
        "world_name": character["world"]
    }
    ocid_data = await fetch_json(session, ocid_url, ocid_params)
    ocid = ocid_data.get("ocid")
    if not ocid:
        print(f"❗ocid 못 찾음 → 닉네임: {character['nickname']} | 월드: {character['world']}")
        return None

    if "-" in character["class"]:
        job, subclass = character["class"].split("-")
    else:
        job = subclass = character["class"]

    if subclass == "전체 전직":
        subclass = job

    result = {
        "nickname": character["nickname"],
        "world": character["world"],
        "level": character["level"],
        "rank": character["rank"],
        "job": job,
        "subclass": subclass
    }

    # character 관련 데이터 수집
    for ep in character_endpoints:
        url = f"https://open.api.nexon.com/maplestory/v1/character/{ep}"
        data = await fetch_json(session, url, {"ocid": ocid, "date": date})
        result[ep] = data

    # user 관련 데이터 수집(유니온)
    for ep in user_endpoints:
        url = f"https://open.api.nexon.com/maplestory/v1/user/{ep}"
        data = await fetch_json(session, url, {"ocid": ocid, "date": date})
        result[ep] = data

    return result

# 클래스별 랭킹 수집
async def gather_rankings_for_class(session, class_name):
    character_list = []
    for page in range(1, 6):  # 페이지 수 조절 가능
        params = {
            "date": date,
            "class": class_name,
            "page": page
        }
        data = await fetch_json(session, "https://open.api.nexon.com/maplestory/v1/ranking/overall", params)
        if not data or "ranking" not in data:
            break
        for entry in data["ranking"]:
            character_list.append({
                "nickname": entry.get("character_name"),
                "world": entry.get("world_name"),
                "level": entry.get("character_level"),
                "rank": entry.get("ranking"),
                "class": class_name
            })
        await asyncio.sleep(0.1)
    return character_list

# 전체 실행 함수
async def main():
    os.makedirs("pirate_json", exist_ok=True) # pirate_json: 폴더명(수정해서 사용해주세요!)
    async with ClientSession() as session:
        for class_name in target_classes:
            print(f"📥 {class_name} 수집 시작")
            character_list = await gather_rankings_for_class(session, class_name)
            results = []
            tasks = [fetch_character_data(session, char) for char in character_list]
            for i in range(0, len(tasks), 50):
                chunk = tasks[i:i + 50]
                chunk_results = await asyncio.gather(*chunk)
                results.extend([r for r in chunk_results if r])
                await asyncio.sleep(0.2)

            df = pd.DataFrame(results)
            safe_class_name = class_name.replace("-", "_")
            df.to_json(f"pirate_json/{safe_class_name}.json", orient="records", force_ascii=False) # pirate_json: 폴더명(수정해서 사용해주세요!)
            print(f"✅ 저장 완료: {safe_class_name}.json")

    return "완료!"

# 실행
await main()

## 데이터가 잘 수집되었는지 확인

In [ ]:
# JSON 불러오기 (기존 코드에서 저장한 경로)
df = pd.read_json("/content/warrior_json/레지스탕스_데몬어벤져.json")

# 상위 5개 행 확인
df.head()

,nickname,world,level,rank,job,subclass,item-equipment,hyper-stat,ability,vmatrix,hexamatrix,symbol-equipment,set-effect,link-skill,hexamatrix-stat,union,union-raider,union-artifact,union-champion
0,으파,오로라,296,1,레지스탕스,데몬어벤져,"{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'ability_gr...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'set_effect...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'union_leve...","{'date': '2025-04-15T00:00+09:00', 'union_raid...","{'date': '2025-04-15T00:00+09:00', 'union_arti...","{'date': '2025-04-15T00:00+09:00', 'union_cham..."
1,검출,베라,296,2,레지스탕스,데몬어벤져,"{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'ability_gr...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'set_effect...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'union_leve...","{'date': '2025-04-15T00:00+09:00', 'union_raid...","{'date': '2025-04-15T00:00+09:00', 'union_arti...","{'date': '2025-04-15T00:00+09:00', 'union_cham..."
2,추돌,스카니아,295,3,레지스탕스,데몬어벤져,"{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'ability_gr...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'set_effect...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'union_leve...","{'date': '2025-04-15T00:00+09:00', 'union_raid...","{'date': '2025-04-15T00:00+09:00', 'union_arti...","{'date': '2025-04-15T00:00+09:00', 'union_cham..."
3,난또삐,스카니아,295,4,레지스탕스,데몬어벤져,"{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'ability_gr...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'set_effect...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'union_leve...","{'date': '2025-04-15T00:00+09:00', 'union_raid...","{'date': '2025-04-15T00:00+09:00', 'union_arti...","{'date': '2025-04-15T00:00+09:00', 'union_cham..."
4,늑몬,엘리시움,295,5,레지스탕스,데몬어벤져,"{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'ability_gr...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'set_effect...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'union_leve...","{'date': '2025-04-15T00:00+09:00', 'union_raid...","{'date': '2025-04-15T00:00+09:00', 'union_arti...","{'date': '2025-04-15T00:00+09:00', 'union_cham..."


In [ ]:
import json

# 첫 번째 row의 item-equipment 열 전체 출력
print(json.dumps(df['item-equipment'][0], indent=2, ensure_ascii=False))

{
  "date": "2025-04-15T00:00+09:00",
  "character_gender": "남",
  "character_class": "데몬어벤져",
  "preset_no": 2,
  "item_equipment": [
    {
      "item_equipment_part": "모자",
      "item_equipment_slot": "모자",
      "item_name": "에테르넬 나이트헬름",
      "item_icon": "https://open.api.nexon.com/static/maplestory/item/icon/KEPCOOGB",
      "item_description": null,
      "item_shape_name": "에테르넬 나이트헬름",
      "item_shape_icon": "https://open.api.nexon.com/static/maplestory/item/icon/KEPCOOGB",
      "item_gender": null,
      "item_total_option": {
        "str": "239",
        "dex": "239",
        "int": "0",
        "luk": "60",
        "max_hp": "13195",
        "max_mp": "0",
        "attack_power": "137",
        "magic_power": "120",
        "armor": "2740",
        "speed": "0",
        "jump": "0",
        "boss_damage": "0",
        "ignore_monster_armor": "15",
        "all_stat": "4",
        "damage": "0",
        "equipment_level_decrease": 0,
        "max_hp_rate": "0",
      

In [ ]:
import json

# 첫 번째 row의 hyper-stat 열 전체 출력
print(json.dumps(df['hyper-stat'][0], indent=2, ensure_ascii=False))

{
  "date": "2025-04-15T00:00+09:00",
  "character_class": "데몬어벤져",
  "use_preset_no": "2",
  "use_available_hyper_stat": 1626,
  "hyper_stat_preset_1": [
    {
      "stat_type": "STR",
      "stat_point": 7,
      "stat_level": 3,
      "stat_increase": "힘 90 증가"
    },
    {
      "stat_type": "DEX",
      "stat_point": null,
      "stat_level": 0,
      "stat_increase": null
    },
    {
      "stat_type": "INT",
      "stat_point": null,
      "stat_level": 0,
      "stat_increase": null
    },
    {
      "stat_type": "LUK",
      "stat_point": null,
      "stat_level": 0,
      "stat_increase": null
    },
    {
      "stat_type": "HP",
      "stat_point": 150,
      "stat_level": 10,
      "stat_increase": "최대 HP 20% 증가"
    },
    {
      "stat_type": "MP",
      "stat_point": null,
      "stat_level": 0,
      "stat_increase": null
    },
    {
      "stat_type": "DF/TF/PP",
      "stat_point": null,
      "stat_level": 0,
      "stat_increase": null
    },
    {
      "stat_

In [ ]:
import json

# 첫 번째 row의 ability 열 전체 출력
print(json.dumps(df['ability'][0], indent=2, ensure_ascii=False))

{
  "date": "2025-04-15T00:00+09:00",
  "ability_grade": "레전드리",
  "ability_info": [
    {
      "ability_no": "1",
      "ability_grade": "레전드리",
      "ability_value": "아이템 드롭률 20% 증가"
    },
    {
      "ability_no": "2",
      "ability_grade": "유니크",
      "ability_value": "일반 몬스터 공격 시 데미지 8% 증가"
    },
    {
      "ability_no": "3",
      "ability_grade": "유니크",
      "ability_value": "메소 획득량 15% 증가"
    }
  ],
  "remain_fame": 3955549,
  "preset_no": 2,
  "ability_preset_1": {
    "ability_preset_grade": "레전드리",
    "ability_info": [
      {
        "ability_no": "1",
        "ability_grade": "레전드리",
        "ability_value": "메소 획득량 20% 증가"
      },
      {
        "ability_no": "2",
        "ability_grade": "유니크",
        "ability_value": "일반 몬스터 공격 시 데미지 8% 증가"
      },
      {
        "ability_no": "3",
        "ability_grade": "유니크",
        "ability_value": "아이템 드롭률 15% 증가"
      }
    ]
  },
  "ability_preset_2": {
    "ability_preset_grade": "레전드리",
    "ability_info": [
 

In [ ]:
import json

# 첫 번째 row의 vmatrix 열 전체 출력
print(json.dumps(df['vmatrix'][0], indent=2, ensure_ascii=False))

{
  "date": "2025-04-15T00:00+09:00",
  "character_class": "데몬어벤져",
  "character_v_core_equipment": [
    {
      "slot_id": "0",
      "slot_level": 5,
      "v_core_name": "추적과 최후의 익시드 : 데몬 스트라이크",
      "v_core_level": 25,
      "v_core_skill_1": "익시드 : 데몬 스트라이크 강화",
      "v_core_skill_2": "실드 체이싱 강화",
      "v_core_skill_3": "인핸스드 익시드 강화",
      "v_core_type": "강화코어"
    },
    {
      "slot_id": "1",
      "slot_level": 5,
      "v_core_name": "최후와 달빛의 사우전드 소드",
      "v_core_level": 25,
      "v_core_skill_1": "사우전드 소드 강화",
      "v_core_skill_2": "인핸스드 익시드 강화",
      "v_core_skill_3": "익시드 : 문라이트 슬래시 강화",
      "v_core_type": "강화코어"
    },
    {
      "slot_id": "2",
      "slot_level": 5,
      "v_core_name": "처형과 천검의 실드 체이싱",
      "v_core_level": 25,
      "v_core_skill_1": "실드 체이싱 강화",
      "v_core_skill_2": "익시드 : 엑스큐션 강화",
      "v_core_skill_3": "사우전드 소드 강화",
      "v_core_type": "강화코어"
    },
    {
      "slot_id": "3",
      "slot_level": 5,
      "v_core_name": "강타와 

In [ ]:
import json

# 첫 번째 row의 hexamatrix 열 전체 출력
print(json.dumps(df['hexamatrix'][0], indent=2, ensure_ascii=False))

{
  "date": "2025-04-15T00:00+09:00",
  "character_hexa_core_equipment": [
    {
      "hexa_core_name": "레퀴엠",
      "hexa_core_level": 30,
      "hexa_core_type": "스킬 코어",
      "linked_skill": [
        {
          "hexa_skill_id": "레퀴엠"
        }
      ]
    },
    {
      "hexa_core_name": "실드 체이싱 VI",
      "hexa_core_level": 30,
      "hexa_core_type": "마스터리 코어",
      "linked_skill": [
        {
          "hexa_skill_id": "실드 체이싱 VI"
        }
      ]
    },
    {
      "hexa_core_name": "익시드 : 엑스큐션 VI",
      "hexa_core_level": 30,
      "hexa_core_type": "마스터리 코어",
      "linked_skill": [
        {
          "hexa_skill_id": "익시드 : 엑스큐션 VI"
        }
      ]
    },
    {
      "hexa_core_name": "익시드 : 문라이트 슬래시 VI/오버맥시마이징 익시드",
      "hexa_core_level": 30,
      "hexa_core_type": "마스터리 코어",
      "linked_skill": [
        {
          "hexa_skill_id": "익시드 : 문라이트 슬래시 VI"
        },
        {
          "hexa_skill_id": "오버맥시마이징 익시드"
        }
      ]
    },
    {
      "hexa_cor

In [ ]:
import json

# 첫 번째 row의 symbol-equipment 열 전체 출력
print(json.dumps(df['symbol-equipment'][0], indent=2, ensure_ascii=False))

{
  "date": "2025-04-15T00:00+09:00",
  "character_class": "데몬어벤져",
  "symbol": [
    {
      "symbol_name": "아케인심볼 : 소멸의 여로",
      "symbol_icon": "https://open.api.nexon.com/static/maplestory/item/icon/KEIDJHOA",
      "symbol_description": "소멸의 여로지역의 아케인심볼이다. 아케인심볼은 아케인포스를 가지며 소지한 자와 공명하여 해당 캐릭터에 맞는 주스탯을 증가시켜준다.\n\n해당 장비를 장착한 후, 인벤토리 내 같은 종류의 아케인심볼을 더블 클릭하면 성장시킬 수 있다. 아케인심볼의 성장치가 일정 수준 이상이 되면 해당 장비를 강화하여 레벨을 올릴 수 있다.\n\n아케인심볼로 상승하는 스탯은 스탯 % 증가 효과를 받지 않는다.",
      "symbol_force": "220",
      "symbol_level": 20,
      "symbol_str": "0",
      "symbol_dex": "0",
      "symbol_int": "0",
      "symbol_luk": "0",
      "symbol_hp": "46200",
      "symbol_drop_rate": "0%",
      "symbol_meso_rate": "0%",
      "symbol_exp_rate": "0%",
      "symbol_growth_count": 0,
      "symbol_require_growth_count": 411
    },
    {
      "symbol_name": "아케인심볼 : 츄츄 아일랜드",
      "symbol_icon": "https://open.api.nexon.com/static/maplestory/item/icon/KEIDJHOD",
      "symbol_description": "츄츄 아일랜드지역의 아케인

In [ ]:
import json

# 첫 번째 row의 set-effect 열 전체 출력
print(json.dumps(df['set-effect'][0], indent=2, ensure_ascii=False))

{
  "date": "2025-04-15T00:00+09:00",
  "set_effect": [
    {
      "set_name": "마이스터 세트",
      "total_set_count": 2,
      "set_effect_info": [
        {
          "set_count": 2,
          "set_option": "최대 HP : +10%, 최대 MP : +10%"
        }
      ],
      "set_option_full": [
        {
          "set_count": 2,
          "set_option": "최대 HP : +10%, 최대 MP : +10%"
        },
        {
          "set_count": 3,
          "set_option": "공격력 : +40, 마력 : +40"
        },
        {
          "set_count": 4,
          "set_option": "보스 몬스터 공격 시 데미지 : +20%"
        }
      ]
    },
    {
      "set_name": "보스 장신구 세트",
      "total_set_count": 2,
      "set_effect_info": [],
      "set_option_full": [
        {
          "set_count": 3,
          "set_option": "올스탯 : +10, 최대 HP : +5%, 최대 MP : +5%, 공격력 : +5, 마력 : +5"
        },
        {
          "set_count": 5,
          "set_option": "올스탯 : +10, 최대 HP : +5%, 최대 MP : +5%, 공격력 : +5, 마력 : +5"
        },
        {
          "set_count": 7,
   

In [ ]:
import json

# 첫 번째 row의 link-skill 열 전체 출력
print(json.dumps(df['link-skill'][0], indent=2, ensure_ascii=False))

{
  "date": "2025-04-15T00:00+09:00",
  "character_class": "데몬어벤져",
  "character_link_skill": [
    {
      "skill_name": "룬 퍼시스턴스",
      "skill_description": "[마스터 레벨 : 2]\r\n룬의 마력을 증폭시켜 해방된 룬의 힘 지속시간을 증가 시킨다.",
      "skill_level": 2,
      "skill_effect": "해방된 룬의 힘 지속시간 50% 증가",
      "skill_icon": "https://open.api.nexon.com/static/maplestory/skill/icon/KFHCLHOCKJ",
      "skill_effect_next": null
    },
    {
      "skill_name": "무아",
      "skill_description": "[마스터 레벨 : 2]\r\n자신을 잊어버릴 정도로 전투에 집중한다.\n일정 시간 동안 전투 상태가 지속되면 데미지가 증가한다.",
      "skill_level": 2,
      "skill_effect": "전투 상태가 5초 지속되면 발동되며 최대 5회 중첩가능, 지속시간 5초\n발동 시 데미지 1% 증가, 중첩당 데미지 2% 추가 증가",
      "skill_icon": "https://open.api.nexon.com/static/maplestory/skill/icon/KFHCLHOENE",
      "skill_effect_next": null
    },
    {
      "skill_name": "아이언 윌",
      "skill_description": "[마스터 레벨 : 2]\r\n카이저는 보다 우월한 HP를 갖는다.",
      "skill_level": 2,
      "skill_effect": "HP 15% 증가",
      "skill_icon": "https://open.api.ne

In [ ]:
import json

# 첫 번째 row의 hexamatrix-stat 열 전체 출력
print(json.dumps(df['hexamatrix-stat'][0], indent=2, ensure_ascii=False))

{
  "date": "2025-04-15T00:00+09:00",
  "character_class": "데몬어벤져",
  "character_hexa_stat_core": [
    {
      "slot_id": "0",
      "main_stat_name": "보스 데미지 증가",
      "sub_stat_name_1": "공격력 증가",
      "sub_stat_name_2": "크리티컬 데미지 증가",
      "main_stat_level": 8,
      "sub_stat_level_1": 6,
      "sub_stat_level_2": 6,
      "stat_grade": 20
    }
  ],
  "character_hexa_stat_core_2": [
    {
      "slot_id": "0",
      "main_stat_name": "공격력 증가",
      "sub_stat_name_1": "크리티컬 데미지 증가",
      "sub_stat_name_2": "보스 데미지 증가",
      "main_stat_level": 8,
      "sub_stat_level_1": 8,
      "sub_stat_level_2": 4,
      "stat_grade": 20
    }
  ],
  "character_hexa_stat_core_3": [
    {
      "slot_id": "0",
      "main_stat_name": "크리티컬 데미지 증가",
      "sub_stat_name_1": "보스 데미지 증가",
      "sub_stat_name_2": "공격력 증가",
      "main_stat_level": 8,
      "sub_stat_level_1": 4,
      "sub_stat_level_2": 8,
      "stat_grade": 20
    }
  ],
  "preset_hexa_stat_core": [
    {
      "slot_id": 

In [ ]:
import json

# 첫 번째 row의 union 열 전체 출력
print(json.dumps(df['union'][0], indent=2, ensure_ascii=False))

{
  "date": "2025-04-15T00:00+09:00",
  "union_level": 9403,
  "union_grade": "그랜드 마스터 유니온 3",
  "union_artifact_level": 56,
  "union_artifact_exp": 14135,
  "union_artifact_point": 18800
}


In [ ]:
import json

# 첫 번째 row의 union-raider 열 전체 출력
print(json.dumps(df['union-raider'][0], indent=2, ensure_ascii=False))

{
  "date": "2025-04-15T00:00+09:00",
  "union_raider_stat": [
    "보스 몬스터 공격 시 데미지 6% 증가",
    "최대 HP 6% 증가",
    "최대 HP 2000 증가",
    "최대 MP 6% 증가",
    "버프 지속시간 25% 증가",
    "STR 80 증가",
    "크리티컬 데미지 6% 증가",
    "크리티컬 확률 5% 증가",
    "공격 시 20%의 확률로 데미지 20% 증가",
    "INT 100 증가",
    "INT 80 증가",
    "공격력/마력 25 증가",
    "STR 80 증가",
    "STR 80 증가",
    "크리티컬 확률 5% 증가",
    "STR 80 증가",
    "INT 80 증가",
    "STR 80 증가",
    "STR 80 증가",
    "경험치 획득량 10% 증가",
    "INT 80 증가",
    "STR 80 증가",
    "DEX 100 증가",
    "DEX 80 증가",
    "최대 HP 2000 증가",
    "메소 획득량 4% 증가",
    "스킬 재사용 대기시간 6% 감소",
    "LUK 80 증가",
    "적 공격마다 70%의 확률로 순수 HP의 10% 회복",
    "LUK 80 증가",
    "INT 80 증가",
    "INT 80 증가",
    "STR, DEX, LUK 40 증가",
    "소환수 지속시간 10% 증가",
    "INT 80 증가",
    "LUK 80 증가",
    "방어율 무시 5% 증가",
    "LUK 80 증가",
    "STR 80 증가"
  ],
  "union_occupied_stat": [
    "획득 경험치 10.0% 증가",
    "INT 5 증가",
    "STR 25 증가",
    "크리티컬 데미지 20.00% 증가",
    "마력 1 증가",
    "최대 HP 3750 증가",
    "크리티

In [ ]:
import json

# 첫 번째 row의 union-artifact 열 전체 출력
print(json.dumps(df['union-artifact'][0], indent=2, ensure_ascii=False))

{
  "date": "2025-04-15T00:00+09:00",
  "union_artifact_effect": [
    {
      "name": "최대 HP 7500, 최대 MP 7500 증가",
      "level": 10
    },
    {
      "name": "공격력 30, 마력 30 증가",
      "level": 10
    },
    {
      "name": "데미지 15.00% 증가",
      "level": 10
    },
    {
      "name": "보스 몬스터 공격 시 데미지 15.00% 증가",
      "level": 10
    },
    {
      "name": "몬스터 방어율 무시 20% 증가",
      "level": 10
    },
    {
      "name": "버프 지속시간 20% 증가",
      "level": 10
    },
    {
      "name": "재사용 대기시간 미적용 확률 7.50% 증가",
      "level": 10
    },
    {
      "name": "메소 획득량 12% 증가",
      "level": 10
    },
    {
      "name": "아이템 드롭률 12% 증가",
      "level": 10
    },
    {
      "name": "크리티컬 확률 20% 증가",
      "level": 10
    },
    {
      "name": "크리티컬 데미지 4.00% 증가",
      "level": 10
    },
    {
      "name": "추가 경험치 획득 12% 증가, 다수 공격 스킬의 최대 공격 가능 대상 수 1 증가",
      "level": 10
    }
  ],
  "union_artifact_crystal": [
    {
      "name": "크리스탈 : 주황버섯",
      "validity_flag": "0",
      "dat

In [ ]:
import json

# 첫 번째 row의 union-champion 열 전체 출력
print(json.dumps(df['union-champion'][0], indent=2, ensure_ascii=False))

{
  "date": "2025-04-15T00:00+09:00",
  "union_champion": [
    {
      "champion_name": "으파",
      "champion_slot": 1,
      "champion_grade": "SSS",
      "champion_class": "데몬어벤져",
      "champion_badge_info": [
        {
          "stat": "올스탯 20, 최대 HP/MP 1000 증가"
        },
        {
          "stat": "공격력/마력 10 증가"
        },
        {
          "stat": "보스 몬스터 공격 시 데미지 5% 증가"
        },
        {
          "stat": "크리티컬 데미지 3.00% 증가"
        },
        {
          "stat": "방어율 무시 5% 증가"
        }
      ]
    },
    {
      "champion_name": "리버에몽",
      "champion_slot": 2,
      "champion_grade": "B",
      "champion_class": "은월",
      "champion_badge_info": [
        {
          "stat": "올스탯 20, 최대 HP/MP 1000 증가"
        }
      ]
    }
  ],
  "champion_badge_total_info": [
    {
      "stat": "올스탯 40, 최대 HP/MP 2000 증가"
    },
    {
      "stat": "공격력/마력 10 증가"
    },
    {
      "stat": "보스 몬스터 공격 시 데미지 5% 증가"
    },
    {
      "stat": "크리티컬 데미지 3.00% 증가"
    },
    {
    

In [ ]:
import json

# 첫 번째 row 기준으로 모든 컬럼 출력
for col in df.columns:
    print(f"\n🟡 [{col}] ------------------------------")

    value = df[col][0]

    # dict나 list 타입이면 보기 좋게 JSON 정렬 출력
    if isinstance(value, (dict, list)):
        print(json.dumps(value, indent=2, ensure_ascii=False))
    else:
        print(value)

## 수집된 데이터 공용 구글 드라이브에 올리기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import shutil
import os

# 로컬에서 작업한 궁수 폴더
local_dir = "/content/archer_json"

# 구글 드라이브의 대상 폴더 (예: 내 드라이브 > maple_data > 궁수)
gdrive_dir = "/content/drive/MyDrive/data/궁수"

# 드라이브 내 폴더 없으면 생성
os.makedirs(gdrive_dir, exist_ok=True)

# 폴더 통째로 복사
for filename in os.listdir(local_dir):
    if filename.endswith(".json"):
        shutil.copy(os.path.join(local_dir, filename), gdrive_dir)

print("✅ 구글 드라이브 업로드 완료!")

✅ 구글 드라이브 업로드 완료!


In [ ]:
import shutil
import os

# 로컬에서 작업한 전사 폴더
local_dir = "/content/warrior_json"

# 구글 드라이브의 대상 폴더 (예: 내 드라이브 > maple_data > 궁수)
gdrive_dir = "/content/drive/MyDrive/data/전사"

# 드라이브 내 폴더 없으면 생성
os.makedirs(gdrive_dir, exist_ok=True)

# 폴더 통째로 복사
for filename in os.listdir(local_dir):
    if filename.endswith(".json"):
        shutil.copy(os.path.join(local_dir, filename), gdrive_dir)

print("✅ 구글 드라이브 업로드 완료!")

✅ 구글 드라이브 업로드 완료!


# json 데이터 살펴보기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os
import glob

# JSON 파일들이 저장된 폴더 경로
json_dir = "/content/drive/MyDrive/data/궁수"

# 모든 JSON 파일 경로 수집 (*.json 파일만)
json_files = glob.glob(os.path.join(json_dir, "*.json"))

# 병합할 데이터프레임 리스트
df_list = []

for file in json_files:
    df = pd.read_json(file)
    df["source_file"] = os.path.basename(file)  # 원본 파일명도 남겨놓기 (선택)
    df_list.append(df)

# 하나의 DataFrame으로 병합
merged_df = pd.concat(df_list, ignore_index=True)

# 결과 확인
print("병합된 총 행 수:", len(merged_df))
print(merged_df.head())

병합된 총 행 수: 6995
      nickname world  level  rank job subclass  \
0        보마노랑이   제니스    299     1  궁수    보우마스터   
1  Marksbowman   유니온    296     2  궁수    보우마스터   
2           폭등  이노시스    295     3  궁수    보우마스터   
3           뽀마   크로아    294     4  궁수    보우마스터   
4           승연    레드    294     5  궁수    보우마스터   

                                      item-equipment  \
0  {'date': '2025-04-15T00:00+09:00', 'character_...   
1  {'date': '2025-04-15T00:00+09:00', 'character_...   
2  {'date': '2025-04-15T00:00+09:00', 'character_...   
3  {'date': '2025-04-15T00:00+09:00', 'character_...   
4  {'date': '2025-04-15T00:00+09:00', 'character_...   

                                          hyper-stat  \
0  {'date': '2025-04-15T00:00+09:00', 'character_...   
1  {'date': '2025-04-15T00:00+09:00', 'character_...   
2  {'date': '2025-04-15T00:00+09:00', 'character_...   
3  {'date': '2025-04-15T00:00+09:00', 'character_...   
4  {'date': '2025-04-15T00:00+09:00', 'character_...   

         

In [ ]:
# JSON 형식으로 저장
merged_df.to_json("/content/drive/MyDrive/data/merged_archer.json", orient="records", force_ascii=False)

In [ ]:
import pandas as pd
import os
import glob

# JSON 파일들이 저장된 폴더 경로
json_dir = "/content/drive/MyDrive/data/도적"

# 모든 JSON 파일 경로 수집 (*.json 파일만)
json_files = glob.glob(os.path.join(json_dir, "*.json"))

# 병합할 데이터프레임 리스트
df_list = []

for file in json_files:
    df = pd.read_json(file)
    df["source_file"] = os.path.basename(file)  # 원본 파일명도 남겨놓기 (선택)
    df_list.append(df)

# 하나의 DataFrame으로 병합
merged_df_thief = pd.concat(df_list, ignore_index=True)

# 결과 확인
print("병합된 총 행 수:", len(merged_df_thief))
print(merged_df_thief.head())

병합된 총 행 수: 8000
  nickname world  level  rank  job subclass  \
0       시가   크로아    296     1  기사단    나이트워커   
1     Fake    베라    296     2  기사단    나이트워커   
2      지봉이    루나    295     3  기사단    나이트워커   
3   향기의아련함  이노시스    295     4  기사단    나이트워커   
4       문구   크로아    294     5  기사단    나이트워커   

                                      item-equipment  \
0  {'date': '2025-04-15T00:00+09:00', 'character_...   
1  {'date': '2025-04-15T00:00+09:00', 'character_...   
2  {'date': '2025-04-15T00:00+09:00', 'character_...   
3  {'date': '2025-04-15T00:00+09:00', 'character_...   
4  {'date': '2025-04-15T00:00+09:00', 'character_...   

                                          hyper-stat  \
0  {'date': '2025-04-15T00:00+09:00', 'character_...   
1  {'date': '2025-04-15T00:00+09:00', 'character_...   
2  {'date': '2025-04-15T00:00+09:00', 'character_...   
3  {'date': '2025-04-15T00:00+09:00', 'character_...   
4  {'date': '2025-04-15T00:00+09:00', 'character_...   

                           

In [ ]:
# JSON 형식으로 저장
merged_df_thief.to_json("/content/drive/MyDrive/data/merged_thief.json", orient="records", force_ascii=False)

In [ ]:
import pandas as pd
import os
import glob

# JSON 파일들이 저장된 폴더 경로
json_dir = "/content/drive/MyDrive/data/마법사"

# 모든 JSON 파일 경로 수집 (*.json 파일만)
json_files = glob.glob(os.path.join(json_dir, "*.json"))

# 병합할 데이터프레임 리스트
df_list = []

for file in json_files:
    df = pd.read_json(file)
    df["source_file"] = os.path.basename(file)  # 원본 파일명도 남겨놓기 (선택)
    df_list.append(df)

# 하나의 DataFrame으로 병합
merged_df_magician = pd.concat(df_list, ignore_index=True)

# 결과 확인
print("병합된 총 행 수:", len(merged_df_magician))
print(merged_df_magician.head())

병합된 총 행 수: 9999
  nickname world  level  rank  job subclass  \
0       솝상   크로아    299     1  마법사       비숍   
1       중뒹  엘리시움    297     2  마법사       비숍   
2       아욤    루나    297     3  마법사       비숍   
3      곰돌이    루나    297     4  마법사       비숍   
4   몽둥이여사님    루나    297     5  마법사       비숍   

                                      item-equipment  \
0  {'date': '2025-04-15T00:00+09:00', 'character_...   
1  {'date': '2025-04-15T00:00+09:00', 'character_...   
2  {'date': '2025-04-15T00:00+09:00', 'character_...   
3  {'date': '2025-04-15T00:00+09:00', 'character_...   
4  {'date': '2025-04-15T00:00+09:00', 'character_...   

                                          hyper-stat  \
0  {'date': '2025-04-15T00:00+09:00', 'character_...   
1  {'date': '2025-04-15T00:00+09:00', 'character_...   
2  {'date': '2025-04-15T00:00+09:00', 'character_...   
3  {'date': '2025-04-15T00:00+09:00', 'character_...   
4  {'date': '2025-04-15T00:00+09:00', 'character_...   

                           

In [ ]:
# JSON 형식으로 저장
merged_df_magician.to_json("/content/drive/MyDrive/data/merged_magician.json", orient="records", force_ascii=False)

In [ ]:
import pandas as pd
import os
import glob

# JSON 파일들이 저장된 폴더 경로
json_dir = "/content/drive/MyDrive/data/전사"

# 모든 JSON 파일 경로 수집 (*.json 파일만)
json_files = glob.glob(os.path.join(json_dir, "*.json"))

# 병합할 데이터프레임 리스트
df_list = []

for file in json_files:
    df = pd.read_json(file)
    df["source_file"] = os.path.basename(file)  # 원본 파일명도 남겨놓기 (선택)
    df_list.append(df)

# 하나의 DataFrame으로 병합
merged_df_warrior = pd.concat(df_list, ignore_index=True)

# 결과 확인
print("병합된 총 행 수:", len(merged_df_warrior))
print(merged_df_warrior.head())

병합된 총 행 수: 11997
  nickname world  level  rank  job subclass  \
0       가람  스카니아    296     1  기사단    소울마스터   
1       영돌  스카니아    296     2  기사단    소울마스터   
2       래찬    루나    295     3  기사단    소울마스터   
3   Center    루나    295     4  기사단    소울마스터   
4       대사    루나    295     5  기사단    소울마스터   

                                      item-equipment  \
0  {'date': '2025-04-15T00:00+09:00', 'character_...   
1  {'date': '2025-04-15T00:00+09:00', 'character_...   
2  {'date': '2025-04-15T00:00+09:00', 'character_...   
3  {'date': '2025-04-15T00:00+09:00', 'character_...   
4  {'date': '2025-04-15T00:00+09:00', 'character_...   

                                          hyper-stat  \
0  {'date': '2025-04-15T00:00+09:00', 'character_...   
1  {'date': '2025-04-15T00:00+09:00', 'character_...   
2  {'date': '2025-04-15T00:00+09:00', 'character_...   
3  {'date': '2025-04-15T00:00+09:00', 'character_...   
4  {'date': '2025-04-15T00:00+09:00', 'character_...   

                          

In [ ]:
# JSON 형식으로 저장
merged_df_warrior.to_json("/content/drive/MyDrive/data/merged_warrior.json", orient="records", force_ascii=False)

In [ ]:
import pandas as pd
import os
import glob

# JSON 파일들이 저장된 폴더 경로
json_dir = "/content/drive/MyDrive/data/해적"

# 모든 JSON 파일 경로 수집 (*.json 파일만)
json_files = glob.glob(os.path.join(json_dir, "*.json"))

# 병합할 데이터프레임 리스트
df_list = []

for file in json_files:
    df = pd.read_json(file)
    df["source_file"] = os.path.basename(file)  # 원본 파일명도 남겨놓기 (선택)
    df_list.append(df)

# 하나의 DataFrame으로 병합
merged_df_pirate = pd.concat(df_list, ignore_index=True)

# 결과 확인
print("병합된 총 행 수:", len(merged_df_pirate))
print(merged_df_pirate.head())

병합된 총 행 수: 7997
  nickname world  level  rank job subclass  \
0       징베   크로아    298     1  해적       캡틴   
1     사선추격  스카니아    295     2  해적       캡틴   
2    학살자펭귄   오로라    295     3  해적       캡틴   
3       럭스    루나    295     4  해적       캡틴   
4      호이퍼  엘리시움    294     5  해적       캡틴   

                                      item-equipment  \
0  {'date': '2025-04-15T00:00+09:00', 'character_...   
1  {'date': '2025-04-15T00:00+09:00', 'character_...   
2  {'date': '2025-04-15T00:00+09:00', 'character_...   
3  {'date': '2025-04-15T00:00+09:00', 'character_...   
4  {'date': '2025-04-15T00:00+09:00', 'character_...   

                                          hyper-stat  \
0  {'date': '2025-04-15T00:00+09:00', 'character_...   
1  {'date': '2025-04-15T00:00+09:00', 'character_...   
2  {'date': '2025-04-15T00:00+09:00', 'character_...   
3  {'date': '2025-04-15T00:00+09:00', 'character_...   
4  {'date': '2025-04-15T00:00+09:00', 'character_...   

                                 

In [ ]:
# JSON 형식으로 저장
merged_df_pirate.to_json("/content/drive/MyDrive/data/merged_pirate.json", orient="records", force_ascii=False)

In [ ]:
import pandas as pd

df = pd.read_json("/content/drive/MyDrive/data/merged_pirate.json")
df.head(1)

,nickname,world,level,rank,job,subclass,item-equipment,hyper-stat,ability,vmatrix,hexamatrix,symbol-equipment,set-effect,link-skill,hexamatrix-stat,union,union-raider,union-artifact,union-champion,source_file
0,징베,크로아,298,1,해적,캡틴,"{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'ability_gr...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'set_effect...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'character_...","{'date': '2025-04-15T00:00+09:00', 'union_leve...","{'date': '2025-04-15T00:00+09:00', 'union_raid...","{'date': '2025-04-15T00:00+09:00', 'union_arti...","{'date': '2025-04-15T00:00+09:00', 'union_cham...",해적_캡틴.json


In [ ]:
import json

# 첫 번째 row 기준으로 모든 컬럼 출력
for col in df.columns:
    print(f"\n🟡 [{col}] ------------------------------")

    value = df[col][0]

    # dict나 list 타입이면 보기 좋게 JSON 정렬 출력
    if isinstance(value, (dict, list)):
        print(json.dumps(value, indent=2, ensure_ascii=False))
    else:
        print(value)


🟡 [nickname] ------------------------------
징베

🟡 [world] ------------------------------
크로아

🟡 [level] ------------------------------
298

🟡 [rank] ------------------------------
1

🟡 [job] ------------------------------
해적

🟡 [subclass] ------------------------------
캡틴

🟡 [item-equipment] ------------------------------
{
  "date": "2025-04-15T00:00+09:00",
  "character_gender": "남",
  "character_class": "캡틴",
  "preset_no": 2,
  "item_equipment": [
    {
      "item_equipment_part": "모자",
      "item_equipment_slot": "모자",
      "item_name": "하이네스 원더러햇",
      "item_icon": "https://open.api.nexon.com/static/maplestory/item/icon/KEPCIPOA",
      "item_description": null,
      "item_shape_name": "가면신사의 모자",
      "item_shape_icon": "https://open.api.nexon.com/static/maplestory/item/icon/KEPCIEGI",
      "item_gender": null,
      "item_total_option": {
        "str": "173",
        "dex": "341",
        "int": "0",
        "luk": "16",
        "max_hp": "2655",
        "max_mp": "36

In [ ]:
df.columns

Index(['nickname', 'world', 'level', 'rank', 'job', 'subclass',
       'item-equipment', 'hyper-stat', 'ability', 'vmatrix', 'hexamatrix',
       'symbol-equipment', 'set-effect', 'link-skill', 'hexamatrix-stat',
       'union', 'union-raider', 'union-artifact', 'union-champion',
       'source_file'],
      dtype='object')

In [ ]:
import json

# 첫 번째 row의 item-equipment 열 전체 출력
print(json.dumps(df['item-equipment'][0], indent=2, ensure_ascii=False))

{
  "date": "2025-04-15T00:00+09:00",
  "character_gender": "남",
  "character_class": "캡틴",
  "preset_no": 2,
  "item_equipment": [
    {
      "item_equipment_part": "모자",
      "item_equipment_slot": "모자",
      "item_name": "하이네스 원더러햇",
      "item_icon": "https://open.api.nexon.com/static/maplestory/item/icon/KEPCIPOA",
      "item_description": null,
      "item_shape_name": "가면신사의 모자",
      "item_shape_icon": "https://open.api.nexon.com/static/maplestory/item/icon/KEPCIEGI",
      "item_gender": null,
      "item_total_option": {
        "str": "173",
        "dex": "341",
        "int": "0",
        "luk": "16",
        "max_hp": "2655",
        "max_mp": "360",
        "attack_power": "88",
        "magic_power": "85",
        "armor": "1425",
        "speed": "0",
        "jump": "0",
        "boss_damage": "0",
        "ignore_monster_armor": "10",
        "all_stat": "7",
        "damage": "0",
        "equipment_level_decrease": 0,
        "max_hp_rate": "0",
        "max_

In [ ]:
import pandas as pd
df = pd.read_csv("sampled_equipment.csv")
df[df['nickname']=='문제인']

,nickname,world,level,job,subclass,equipment_part,equipment_slot,item_name,starforce,str_total,...,special_ring_level,growth_exp,growth_level,scroll_upgrade,cuttable_count,golden_hammer_flag,scroll_resilience_count,scroll_upgradeable_count,soul_name,soul_option
0,문제인,루나,295,칼리,칼리,모자,모자,앱솔랩스 시프캡,18,0,...,NaN,0,0,12,3,적용,0,0,NaN,NaN
1,문제인,루나,295,칼리,칼리,얼굴장식,얼굴장식,트와일라이트 마크,18,92,...,NaN,0,0,4,7,적용,0,0,NaN,NaN
2,문제인,루나,295,칼리,칼리,눈장식,눈장식,파풀라투스 마크,18,75,...,NaN,0,0,6,9,적용,0,0,NaN,NaN
3,문제인,루나,295,칼리,칼리,귀고리,귀고리,에스텔라 이어링,18,165,...,NaN,0,0,7,6,적용,0,0,NaN,NaN
4,문제인,루나,295,칼리,칼리,상의,상의,이글아이 어새신셔츠,18,0,...,NaN,0,0,8,4,적용,0,0,NaN,NaN
5,문제인,루나,295,칼리,칼리,하의,하의,트릭스터 어새신팬츠,18,40,...,NaN,0,0,8,6,적용,0,0,NaN,NaN
6,문제인,루나,295,칼리,칼리,신발,신발,앱솔랩스 시프슈즈,18,85,...,NaN,0,0,8,4,적용,0,0,NaN,NaN
7,문제인,루나,295,칼리,칼리,장갑,장갑,앱솔랩스 시프글러브,18,30,...,NaN,0,0,8,3,적용,0,0,NaN,NaN
8,문제인,루나,295,칼리,칼리,망토,망토,앱솔랩스 시프케이프,18,76,...,NaN,0,0,8,6,적용,0,0,NaN,NaN
9,문제인,루나,295,칼리,칼리,헥스시커,보조무기,인피니트 헥스시커,0,0,...,NaN,0,0,0,255,미적용,0,0,NaN,NaN


In [ ]:
df.columns

Index(['nickname', 'world', 'level', 'job', 'subclass', 'equipment_part',
       'equipment_slot', 'item_name', 'starforce', 'str_total',
       ...
       'special_ring_level', 'growth_exp', 'growth_level', 'scroll_upgrade',
       'cuttable_count', 'golden_hammer_flag', 'scroll_resilience_count',
       'scroll_upgradeable_count', 'soul_name', 'soul_option'],
      dtype='object', length=106)